In [0]:
# Define o caminho do arquivo
file_path = 'dbfs:/Volumes/workspace/default/veiculos/dados_veiculos_2025.csv'

# Lê o arquivo CSV como DataFrame Spark, com opções de cabeçalho, inferência de schema, delimitador e encoding
df = (
    spark.read.format('csv')
    .option('header', 'true')
    .option('inferSchema', 'true')
    .option('delimiter', ';')
    .option('encoding', 'ISO-8859-1')
    .load(file_path)
)

from pyspark.sql.functions import regexp_replace, col, sum, count, udf
from pyspark.sql.types import StringType

# Substitui vírgula por ponto e converte a coluna Valor_Venda para float
df = df.withColumn(
    'Valor_Venda',
    regexp_replace('Valor_Venda', ',', '.').cast('float')
)

# Função para formatar valores como moeda brasileira
def formatar_moeda(valor):
    if valor is None:
        return None
    return f"R$ {valor:,.2f}".replace(",", "X").replace(".", ",").replace("X", ".")

# Cria UDF para aplicar a formatação de moeda
formatar_moeda_udf = udf(formatar_moeda, StringType())

# Agrupa por Funcionario e Ano, calcula soma e quantidade de vendas, e formata o total de vendas
df_funcionario = (
    df.groupBy('Funcionario', 'Ano')
    .agg(
        sum('Valor_Venda').alias('Total_Vendas'),
        count('*').alias('Quantidade_Vendas')
    )
    .withColumn(
        "Total_Vendas_Formatado",
        formatar_moeda_udf(col("Total_Vendas"))
    )
    .select('Funcionario', 'Ano', 'Quantidade_Vendas', 'Total_Vendas_Formatado')
)

# Exibe o resultado formatado
display(df_funcionario)

Funcionario,Ano,Quantidade_Vendas,Total_Vendas_Formatado
Mariana Costa,2002,355,"R$ 23.717.135,07"
Carlos Silva,2007,336,"R$ 24.221.280,53"
Pedro Rocha,2010,333,"R$ 22.850.010,02"
Carlos Silva,2000,335,"R$ 23.629.867,61"
Fernanda Alves,2016,398,"R$ 27.871.271,73"
Pedro Rocha,2015,344,"R$ 24.656.177,18"
Carlos Silva,2016,339,"R$ 23.557.139,20"
João Lima,2023,351,"R$ 24.212.236,89"
Mariana Costa,2023,352,"R$ 24.451.859,31"
Pedro Rocha,2014,356,"R$ 24.475.686,03"
